In [2]:
import pandas as pd
import pickle
import argparse
import os
import glob
import pydicom
import sys
#from Code.volumetryworker.main import run_volumetry
#from Code.wmhypoworker.main import run_wmh
#from Code.Report_code.generate_reports import atrophy_report, wmh_report, patient_report



/tmp/ipykernel_1766286/959883630.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [3]:
combined_roi_dict = {
    'Left-Cingulate': {
        'ctx-lh-caudalanteriorcingulate',
        'ctx-lh-rostralanteriorcingulate',
        'ctx-lh-isthmuscingulate',
        'ctx-lh-posteriorcingulate',
    },
    'Right-Cingulate': {
        'ctx-rh-caudalanteriorcingulate',
        'ctx-rh-rostralanteriorcingulate',
        'ctx-rh-isthmuscingulate',
        'ctx-rh-posteriorcingulate',
    },
    'Left-Frontal': {
        'ctx-lh-caudalmiddlefrontal',
        'ctx-lh-lateralorbitofrontal',
        'ctx-lh-medialorbitofrontal',
        'ctx-lh-parsopercularis',
        'ctx-lh-parsorbitalis',
        'ctx-lh-parstriangularis',
        'ctx-lh-precentral',
        'ctx-lh-rostralmiddlefrontal',
        'ctx-lh-superiorfrontal',
        'ctx-lh-paracentral'
    },
    'Right-Frontal': {
        'ctx-rh-caudalmiddlefrontal',
        'ctx-rh-lateralorbitofrontal',
        'ctx-rh-medialorbitofrontal',
        'ctx-rh-parsopercularis',
        'ctx-rh-parsorbitalis',
        'ctx-rh-parstriangularis',
        'ctx-rh-precentral',
        'ctx-rh-rostralmiddlefrontal',
        'ctx-rh-superiorfrontal',
        'ctx-rh-paracentral'
    },
    'Left-Temporal': {
        'ctx-lh-entorhinal',
        'ctx-lh-fusiform',
        'ctx-lh-inferiortemporal',
        'ctx-lh-middletemporal',
        'ctx-lh-superiortemporal',
        'ctx-lh-transversetemporal',
        'ctx-lh-parahippocampal',
        'Left-Hippocampus',
        'Left-Amygdala',
    },
    'Right-Temporal': {
        'ctx-rh-entorhinal',
        'ctx-rh-fusiform',
        'ctx-rh-inferiortemporal',
        'ctx-rh-middletemporal',
        'ctx-rh-superiortemporal',
        'ctx-rh-transversetemporal',
        'ctx-rh-parahippocampal',
        'Right-Hippocampus',
        'Right-Amygdala',
    },
    'Left-Parietal': {
        'ctx-lh-inferiorparietal',
        'ctx-lh-postcentral',
        'ctx-lh-superiorparietal',
        'ctx-lh-supramarginal',
        'ctx-lh-precuneus'
    },
    'Right-Parietal': {
        'ctx-rh-inferiorparietal',
        'ctx-rh-postcentral',
        'ctx-rh-superiorparietal',
        'ctx-rh-supramarginal',
        'ctx-rh-precuneus'
    },
    'Left-Occipital': {
        'ctx-lh-cuneus',
        'ctx-lh-lateraloccipital',
        'ctx-lh-lingual',
        'ctx-lh-pericalcarine'
    },
    'Right-Occipital': {
        'ctx-rh-cuneus',
        'ctx-rh-lateraloccipital',
        'ctx-rh-lingual',
        'ctx-rh-pericalcarine'
    },
    'Left-Insula':{
        'ctx-lh-insula'
    },
    'Right-Insula': {
        'ctx-rh-insula'
    },
    'Left-Cerebellum':{
        'Left-Cerebellum-White-Matter',
        'Left-Cerebellum-Cortex'
    },
    'Right-Cerebellum': {
        'Right-Cerebellum-White-Matter',
        'Right-Cerebellum-Cortex'
    },
    'Left-Combined-Cerebral-WM': {
        'Left-Cerebral-White-Matter',
        'Left-WM-hypointensities'
    },
    'Right-Combined-Cerebral-WM': {
        'Right-Cerebral-White-Matter',
        'Right-WM-hypointensities'
    },
    'Left-Total-Lateral-Ventricle': {
        'Left-Lateral-Ventricle',
        'Left-Inf-Lat-Vent'
    },
    'Right-Total-Lateral-Ventricle': {
        'Right-Lateral-Ventricle',
        'Right-Inf-Lat-Vent'
    },
}
gray_matter_index = {
    'left_cortical_gm': {
        'left_cingulate',
        'left_frontal',
        'left_temporal',
        'left_parietal',
        'left_occipital',
        'left_insula'
    },
    'right_cortical_gm': {
        'right_cingulate',
        'right_frontal',
        'right_temporal',
        'right_parietal',
        'right_occipital',
        'right_insula'
    }
}
additional_index = [
    'whole_brain',
    'cortical_gm',
    'combined_cerebral_wm',
    'frontal',
    'temporal',
    'parietal',
    'occipital',
    'insula',
    'thalamus',
    'caudate',
    'putamen',
    'pallidum',
    'hippocampus',
    'amygdala',
    'accumbens_area',
    'ventraldc',
    'cingulate',
    'cerebellum',
    'cerebellum_white_matter',
    'cerebellum_cortex',
    'lateral_ventricle',
    'inf_lat_vent',
    'total_lateral_ventricle',
    'wm_hypointensities',
    'HippocampusOccupancyScore',
    'HippocampusAsymmetryIndex',
]

combined_roi_display_dict = {
    'Left-Cingulate': {
        'ctx-lh-caudalanteriorcingulate',
        'ctx-lh-rostralanteriorcingulate',
        'ctx-lh-isthmuscingulate',
        'ctx-lh-posteriorcingulate',
    },
    'Right-Cingulate': {
        'ctx-rh-caudalanteriorcingulate',
        'ctx-rh-rostralanteriorcingulate',
        'ctx-rh-isthmuscingulate',
        'ctx-rh-posteriorcingulate',
    },
    'Left-Frontal': {
        'ctx-lh-caudalmiddlefrontal',
        'ctx-lh-lateralorbitofrontal',
        'ctx-lh-medialorbitofrontal',
        'ctx-lh-parsopercularis',
        'ctx-lh-parsorbitalis',
        'ctx-lh-parstriangularis',
        'ctx-lh-precentral',
        'ctx-lh-rostralmiddlefrontal',
        'ctx-lh-superiorfrontal',
        'ctx-lh-paracentral'
    },
    'Right-Frontal': {
        'ctx-rh-caudalmiddlefrontal',
        'ctx-rh-lateralorbitofrontal',
        'ctx-rh-medialorbitofrontal',
        'ctx-rh-parsopercularis',
        'ctx-rh-parsorbitalis',
        'ctx-rh-parstriangularis',
        'ctx-rh-precentral',
        'ctx-rh-rostralmiddlefrontal',
        'ctx-rh-superiorfrontal',
        'ctx-rh-paracentral'
    },
    'Left-Temporal': {
        'ctx-lh-entorhinal',
        'ctx-lh-fusiform',
        'ctx-lh-inferiortemporal',
        'ctx-lh-middletemporal',
        'ctx-lh-superiortemporal',
        'ctx-lh-transversetemporal',
        'ctx-lh-parahippocampal',
    },
    'Right-Temporal': {
        'ctx-rh-entorhinal',
        'ctx-rh-fusiform',
        'ctx-rh-inferiortemporal',
        'ctx-rh-middletemporal',
        'ctx-rh-superiortemporal',
        'ctx-rh-transversetemporal',
        'ctx-rh-parahippocampal',
    },
    'Left-Parietal': {
        'ctx-lh-inferiorparietal',
        'ctx-lh-postcentral',
        'ctx-lh-superiorparietal',
        'ctx-lh-supramarginal',
        'ctx-lh-precuneus'
    },
    'Right-Parietal': {
        'ctx-rh-inferiorparietal',
        'ctx-rh-postcentral',
        'ctx-rh-superiorparietal',
        'ctx-rh-supramarginal',
        'ctx-rh-precuneus'
    },
    'Left-Occipital': {
        'ctx-lh-cuneus',
        'ctx-lh-lateraloccipital',
        'ctx-lh-lingual',
        'ctx-lh-pericalcarine'
    },
    'Right-Occipital': {
        'ctx-rh-cuneus',
        'ctx-rh-lateraloccipital',
        'ctx-rh-lingual',
        'ctx-rh-pericalcarine'
    },
    'Left-Insula':{
        'ctx-lh-insula'
    },
    'Right-Insula': {
        'ctx-rh-insula'
    },
    'Left-Cerebellum':{
        'Left-Cerebellum-White-Matter',
        'Left-Cerebellum-Cortex'
    },
    'Right-Cerebellum': {
        'Right-Cerebellum-White-Matter',
        'Right-Cerebellum-Cortex'
    },
    'Left-Combined-Cerebral-WM': {
        'Left-Cerebral-White-Matter',
        'Left-WM-hypointensities'
    },
    'Right-Combined-Cerebral-WM': {
        'Right-Cerebral-White-Matter',
        'Right-WM-hypointensities'
    }
}


In [4]:
def load_pickle_file(filepath):
    with open(filepath, 'rb') as file:
        data = pickle.load(file)
    return data
# 파일열어보기 , PICKLE로 되어있는것

In [5]:
volumes_path = '/home/limseoyoung/drwelly1/SeoyoungLim/drwelly-debug/drwelly_oasis2_before_swift/OAS2_0186_MR3_mpr-3_Tag_pr100_sr100_ppf100_spf100_ipf100_sipf179/111111/Volumetry/volumes.pickle'

volumes_data = load_pickle_file(volumes_path)

In [6]:
volumes_data 

{'total_intracranial_volume': 1317455.0436916205,
 'unknown': 15499112.425155036,
 'left_cerebral_white_matter': 182121.6716954589,
 'left_lateral_ventricle': 11248.341425325641,
 'left_inf_lat_vent': 491.86197487092056,
 'left_cerebellum_white_matter': 13399.980315399062,
 'left_cerebellum_cortex': 57534.47406058113,
 'left_thalamus': 6195.265683485751,
 'left_caudate': 3555.1948183661725,
 'left_putamen': 3708.1728105505736,
 'left_pallidum': 1640.5689161838307,
 '3rd_ventricle': 1246.8049363011132,
 '4th_ventricle': 1696.477913327457,
 'brain_stem': 19779.093989492074,
 'left_hippocampus': 3164.517838325741,
 'left_amygdala': 1331.5259319727431,
 'csf': 1061.2419457814701,
 'left_accumbens_area': 448.643977078915,
 'left_ventraldc': 3785.347806607726,
 'right_cerebral_white_matter': 181217.86674163403,
 'right_lateral_ventricle': 9506.930514293797,
 'right_inf_lat_vent': 466.8229761501554,
 'right_cerebellum_white_matter': 13596.176305375468,
 'right_cerebellum_cortex': 58694.157001

In [7]:
import os
import pandas as pd
import pickle

def load_pickle_file(file_path):
    with open(file_path, 'rb') as file:
        return pickle.load(file)
        
def find_volumes_files(root_path):
    volumes_files = []
    for root, dirs, files in os.walk(root_path):
        for file in files:
            if file == 'volumes.pickle':
                volumes_files.append(os.path.join(root, file))
    return volumes_files

def extract_subject_data(volumes_file_path):
    subject_name = os.path.dirname(volumes_file_path).split(os.sep)[-3]
    volumes_data = load_pickle_file(volumes_file_path)
    volumes_data['subject'] = subject_name
    return volumes_data

def compile_data(volumes_files):
    data = []
    for volumes_file in volumes_files:
        subject_data = extract_subject_data(volumes_file)
        data.append(subject_data)
    df = pd.DataFrame(data)
    # You might need to adjust the DataFrame to ensure it has the structure you want
    return df

root_path = '/home/limseoyoung/drwelly1/SeoyoungLim/drwelly-debug/drwelly_oasis2_before_swift'
volumes_files = find_volumes_files(root_path)
df = compile_data(volumes_files)


In [8]:
df

,total_intracranial_volume,unknown,left_cerebral_white_matter,left_lateral_ventricle,left_inf_lat_vent,left_cerebellum_white_matter,left_cerebellum_cortex,left_thalamus,left_caudate,left_putamen,...,cerebellum,cerebellum_white_matter,cerebellum_cortex,lateral_ventricle,inf_lat_vent,total_lateral_ventricle,wm_hypointensities,HippocampusOccupancyScore,HippocampusAsymmetryIndex,subject
0,1.358897e+06,1.545767e+07,183010.727650,8594.893561,866.760956,9603.999509,44477.150728,6575.995664,2750.516859,4793.424755,...,106929.215537,18798.114040,88131.101497,16437.931160,1676.926914,18114.858075,1229.997937,0.788825,0.020086,OAS2_0174_MR3_mpr-2_Tag_pr100_sr100_ppf100_spf...
1,1.783869e+06,1.503270e+07,252787.557085,15076.564230,1219.707938,12931.785339,55145.136183,7858.472599,4185.628786,5293.175730,...,138131.237943,25875.232678,112256.005265,31510.722390,2368.757879,33879.480269,3632.712814,0.779730,0.005155,OAS2_0171_MR1_mpr-1_Tag_pr100_sr100_ppf100_spf...
2,1.715428e+06,1.510114e+07,226738.423416,30430.615445,2250.079885,13326.921319,50958.135397,8036.146589,3726.694810,5298.320729,...,127998.675461,25965.784673,102032.890787,63909.471735,4158.874788,68068.346522,4924.107748,0.648719,0.022363,OAS2_0175_MR1_mpr-3_Tag_pr100_sr100_ppf100_spf...
3,1.620240e+06,1.519633e+07,202643.703647,30157.930459,1520.861922,9846.157497,45718.810664,6624.015662,3024.230845,4675.089761,...,112441.225255,19451.186006,92990.039249,63302.018766,3342.534829,66644.553595,9640.357507,0.666633,0.026868,OAS2_0185_MR1_mpr-1_Tag_pr100_sr100_ppf100_spf...
4,1.701519e+06,1.511505e+07,222716.748621,32542.123337,2617.432866,11711.734402,51264.434381,7800.162601,3858.406803,5319.929728,...,125314.357598,23159.701817,102154.655781,67646.456544,4696.355760,72342.812304,8218.279580,0.611019,0.033923,OAS2_0175_MR2_mpr-4_Tag_pr100_sr100_ppf100_spf...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
407,1.440044e+06,1.537652e+07,179055.937852,23762.009786,1642.626916,8751.301553,43933.495755,5205.710734,3017.027846,4122.859789,...,103575.018708,16930.479135,86644.539573,43948.930755,2862.677854,46811.608608,7125.824636,0.694184,0.033335,OAS2_0080_MR1_mpr-1_Tag_pr100_sr100_ppf100_spf...
408,1.456309e+06,1.536026e+07,189033.121342,24576.977744,1603.524918,10630.941457,44038.110750,6642.537661,4183.913786,4669.258761,...,108759.806443,20675.352944,88084.453500,48239.174535,3038.293845,51277.468380,18740.833043,0.662050,0.016318,OAS2_0178_MR3_mpr-3_Tag_pr100_sr100_ppf100_spf...
409,1.392358e+06,1.542421e+07,187889.559401,30218.984456,1966.075900,15542.358206,48941.638500,6614.068662,3645.060814,4060.433793,...,127257.452498,30303.362452,96954.090047,54792.189201,4205.179785,58997.368986,8241.946579,0.627158,0.071539,OAS2_0046_MR1_mpr-3_Tag_pr100_sr100_ppf100_spf...
410,1.317240e+06,1.549933e+07,174959.832061,13979.993286,1033.801947,9140.949533,37935.455062,5862.212701,2812.942856,3904.711801,...,95731.981109,18075.756077,77656.225033,27322.006604,1952.012900,29274.019504,2830.092855,0.770967,0.027075,OAS2_0027_MR3_mpr-1_Tag_pr100_sr100_ppf100_spf...


In [9]:
df['subject'] = df['subject'].str.replace('_Tag_pr100_sr100_ppf100_spf100_ipf100_sipf179', '')


In [10]:
df

,total_intracranial_volume,unknown,left_cerebral_white_matter,left_lateral_ventricle,left_inf_lat_vent,left_cerebellum_white_matter,left_cerebellum_cortex,left_thalamus,left_caudate,left_putamen,...,cerebellum,cerebellum_white_matter,cerebellum_cortex,lateral_ventricle,inf_lat_vent,total_lateral_ventricle,wm_hypointensities,HippocampusOccupancyScore,HippocampusAsymmetryIndex,subject
0,1.358897e+06,1.545767e+07,183010.727650,8594.893561,866.760956,9603.999509,44477.150728,6575.995664,2750.516859,4793.424755,...,106929.215537,18798.114040,88131.101497,16437.931160,1676.926914,18114.858075,1229.997937,0.788825,0.020086,OAS2_0174_MR3_mpr-2
1,1.783869e+06,1.503270e+07,252787.557085,15076.564230,1219.707938,12931.785339,55145.136183,7858.472599,4185.628786,5293.175730,...,138131.237943,25875.232678,112256.005265,31510.722390,2368.757879,33879.480269,3632.712814,0.779730,0.005155,OAS2_0171_MR1_mpr-1
2,1.715428e+06,1.510114e+07,226738.423416,30430.615445,2250.079885,13326.921319,50958.135397,8036.146589,3726.694810,5298.320729,...,127998.675461,25965.784673,102032.890787,63909.471735,4158.874788,68068.346522,4924.107748,0.648719,0.022363,OAS2_0175_MR1_mpr-3
3,1.620240e+06,1.519633e+07,202643.703647,30157.930459,1520.861922,9846.157497,45718.810664,6624.015662,3024.230845,4675.089761,...,112441.225255,19451.186006,92990.039249,63302.018766,3342.534829,66644.553595,9640.357507,0.666633,0.026868,OAS2_0185_MR1_mpr-1
4,1.701519e+06,1.511505e+07,222716.748621,32542.123337,2617.432866,11711.734402,51264.434381,7800.162601,3858.406803,5319.929728,...,125314.357598,23159.701817,102154.655781,67646.456544,4696.355760,72342.812304,8218.279580,0.611019,0.033923,OAS2_0175_MR2_mpr-4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
407,1.440044e+06,1.537652e+07,179055.937852,23762.009786,1642.626916,8751.301553,43933.495755,5205.710734,3017.027846,4122.859789,...,103575.018708,16930.479135,86644.539573,43948.930755,2862.677854,46811.608608,7125.824636,0.694184,0.033335,OAS2_0080_MR1_mpr-1
408,1.456309e+06,1.536026e+07,189033.121342,24576.977744,1603.524918,10630.941457,44038.110750,6642.537661,4183.913786,4669.258761,...,108759.806443,20675.352944,88084.453500,48239.174535,3038.293845,51277.468380,18740.833043,0.662050,0.016318,OAS2_0178_MR3_mpr-3
409,1.392358e+06,1.542421e+07,187889.559401,30218.984456,1966.075900,15542.358206,48941.638500,6614.068662,3645.060814,4060.433793,...,127257.452498,30303.362452,96954.090047,54792.189201,4205.179785,58997.368986,8241.946579,0.627158,0.071539,OAS2_0046_MR1_mpr-3
410,1.317240e+06,1.549933e+07,174959.832061,13979.993286,1033.801947,9140.949533,37935.455062,5862.212701,2812.942856,3904.711801,...,95731.981109,18075.756077,77656.225033,27322.006604,1952.012900,29274.019504,2830.092855,0.770967,0.027075,OAS2_0027_MR3_mpr-1


In [11]:
# 'mpr-x'에 따라 4개의 데이터프레임으로 분할
volume_df_mpr1 = df[df['subject'].str.contains('MR1_mpr-1')]
volume_df_mpr2 = df[df['subject'].str.contains('MR1_mpr-2')]
volume_df_mpr3 = df[df['subject'].str.contains('MR1_mpr-3')]
volume_df_mpr4 = df[df['subject'].str.contains('MR1_mpr-4')]

In [12]:
volume_df_mpr1

,total_intracranial_volume,unknown,left_cerebral_white_matter,left_lateral_ventricle,left_inf_lat_vent,left_cerebellum_white_matter,left_cerebellum_cortex,left_thalamus,left_caudate,left_putamen,...,cerebellum,cerebellum_white_matter,cerebellum_cortex,lateral_ventricle,inf_lat_vent,total_lateral_ventricle,wm_hypointensities,HippocampusOccupancyScore,HippocampusAsymmetryIndex,subject
1,1.783869e+06,1.503270e+07,252787.557085,15076.564230,1219.707938,12931.785339,55145.136183,7858.472599,4185.628786,5293.175730,...,138131.237943,25875.232678,112256.005265,31510.722390,2368.757879,33879.480269,3632.712814,0.779730,0.005155,OAS2_0171_MR1_mpr-1
3,1.620240e+06,1.519633e+07,202643.703647,30157.930459,1520.861922,9846.157497,45718.810664,6624.015662,3024.230845,4675.089761,...,112441.225255,19451.186006,92990.039249,63302.018766,3342.534829,66644.553595,9640.357507,0.666633,0.026868,OAS2_0185_MR1_mpr-1
5,1.440649e+06,1.537592e+07,161264.528761,19763.658990,1865.233905,10172.693480,45737.332663,6079.331689,3181.324837,4389.713776,...,110621.610348,19685.454994,90936.155354,33421.918292,3389.868827,36811.787119,2378.704878,0.642860,0.033286,OAS2_0165_MR1_mpr-1
19,1.608101e+06,1.520847e+07,203166.778620,36894.450115,1985.969899,12120.933381,43074.280799,6884.009648,3661.524813,3846.058804,...,111280.856315,24280.625760,87000.230555,64950.133682,3683.819812,68633.953494,44501.503726,0.641812,0.032503,OAS2_0164_MR1_mpr-1
21,1.322464e+06,1.549410e+07,185442.597526,9640.700507,397.536980,13101.913331,57709.747052,6395.234673,3492.425822,3711.259810,...,142924.662698,26256.648659,116668.014039,17920.377084,843.093957,18763.471041,1097.599944,0.886913,0.011979,OAS2_0186_MR1_mpr-1
33,1.534214e+06,1.528235e+07,210800.243230,31177.326407,1450.203926,10991.777438,47181.705590,6747.495655,3268.446833,4255.943783,...,117665.800988,21608.998896,96056.802092,77831.841024,3376.148828,81207.989851,2674.027863,0.616131,0.008991,OAS2_0182_MR1_mpr-1
47,1.385521e+06,1.543105e+07,196618.222955,20922.312931,775.865960,11935.713390,49614.604465,6318.059677,3262.958833,4306.021780,...,122700.697731,23918.760778,98781.936953,42469.228830,1709.511913,44178.740743,3916.716800,0.785841,0.015159,OAS2_0183_MR1_mpr-1
55,1.511523e+06,1.530504e+07,202602.200649,20756.300940,1112.691943,12812.421345,53764.904253,7032.528641,3387.467827,4933.368748,...,131013.645307,24753.279735,106260.365571,44720.337715,2179.078889,46899.416604,1981.853899,0.765861,0.020115,OAS2_0061_MR1_mpr-1
59,1.306255e+06,1.551031e+07,188102.219390,13050.463333,671.593966,13358.477318,46369.481631,6754.012655,3360.027828,4556.411767,...,119883.295875,26014.147671,93869.148204,29771.369479,1312.317933,31083.687412,3055.100844,0.838066,0.031776,OAS2_0029_MR1_mpr-1
72,1.556886e+06,1.525968e+07,223946.746559,16883.831137,1236.171937,12453.986364,48029.258546,6397.978673,2881.542853,4294.702781,...,122360.784749,24723.781737,97637.003012,36628.625129,2554.320870,39182.945998,2373.559879,0.706285,0.037649,OAS2_0108_MR1_mpr-1


In [16]:
#volume_df_mpr1.to_csv("DRWELLY_mr1_for_tabnet.csv") 

In [17]:
#volume_df_mpr2.to_csv("DRWELLY_mr2_for_tabnet.csv") 

In [13]:

def extract_number(subj):
    parts = subj.split('_')  # '_'를 기준으로 문자열 분리
    if len(parts) > 1:
        number_part = parts[1]  # "OAS1_" 다음 오는 부분을 선택
        # 숫자만 추출
        numbers = ''.join(filter(str.isdigit, number_part))
        return int(numbers) if numbers.isdigit() else None
    return None

In [14]:

volume_df_mpr1 ['sort_key'] = volume_df_mpr1['subject'].apply(extract_number)  # 정렬 키 생성
volume_df_sorted1 = volume_df_mpr1 .sort_values(by='sort_key').drop(columns=['sort_key'])  # 정렬 및 정렬 키 컬럼 삭제


/tmp/ipykernel_1766286/3245932642.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  volume_df_mpr1 ['sort_key'] = volume_df_mpr1['subject'].apply(extract_number)  # 정렬 키 생성


In [15]:

volume_df_mpr2 ['sort_key'] = volume_df_mpr2['subject'].apply(extract_number)  # 정렬 키 생성
volume_df_sorted2 = volume_df_mpr2.sort_values(by='sort_key').drop(columns=['sort_key'])  # 정렬 및 정렬 키 컬럼 삭제


/tmp/ipykernel_1766286/4065747069.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  volume_df_mpr2 ['sort_key'] = volume_df_mpr2['subject'].apply(extract_number)  # 정렬 키 생성


In [16]:

volume_df_mpr3['sort_key'] = volume_df_mpr3['subject'].apply(extract_number)  # 정렬 키 생성
volume_df_sorted3 = volume_df_mpr3.sort_values(by='sort_key').drop(columns=['sort_key'])  # 정렬 및 정렬 키 컬럼 삭제


/tmp/ipykernel_1766286/593579216.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  volume_df_mpr3['sort_key'] = volume_df_mpr3['subject'].apply(extract_number)  # 정렬 키 생성


In [17]:
volume_df_mpr4['sort_key'] = volume_df_mpr4['subject'].apply(extract_number)  # 정렬 키 생성
volume_df_sorted4 = volume_df_mpr4.sort_values(by='sort_key').drop(columns=['sort_key'])  # 정렬 및 정렬 키 컬럼 삭제


/tmp/ipykernel_1766286/3330656496.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  volume_df_mpr4['sort_key'] = volume_df_mpr4['subject'].apply(extract_number)  # 정렬 키 생성


In [18]:
import numpy as np

In [19]:
np.random.seed(0) 

In [25]:

brain_areas = set()
left_right_columns = []

for col in volume_df_sorted1.columns:
    if col.startswith('left_') or col.startswith('right_'):
        left_right_columns.append(col)
        brain_area = col.split('_', 1)[1]
        brain_areas.add(brain_area)


for area in brain_areas:
    left_col = f'left_{area}'
    right_col = f'right_{area}'
    volume_df_sorted1[area] = volume_df_sorted1.get(left_col, 0) + volume_df_mpr1.get(right_col, 0)


volume_df_sorted1 =  volume_df_sorted1.drop(columns=[col for col in left_right_columns])


In [27]:
v#olume_df_sorted1.to_csv("DRWELLY_mr1_for_tabnet222.csv") 

In [28]:

brain_areas = set()
left_right_columns = []

for col in volume_df_sorted1.columns:
    if col.startswith('left_') or col.startswith('right_'):
        left_right_columns.append(col)
        brain_area = col.split('_', 1)[1]
        brain_areas.add(brain_area)


for area in brain_areas:
    left_col = f'left_{area}'
    right_col = f'right_{area}'
    volume_df_sorted2[area] = volume_df_sorted2.get(left_col, 0) + volume_df_mpr2.get(right_col, 0)


volume_df_sorted2 =  volume_df_sorted2.drop(columns=[col for col in left_right_columns])


In [29]:
#volume_df_sorted2.to_csv("DRWELLY_mr2_for_tabnet222.csv") 

In [20]:
volume_df_mpr1 

,total_intracranial_volume,unknown,left_cerebral_white_matter,left_lateral_ventricle,left_inf_lat_vent,left_cerebellum_white_matter,left_cerebellum_cortex,left_thalamus,left_caudate,left_putamen,...,cerebellum_white_matter,cerebellum_cortex,lateral_ventricle,inf_lat_vent,total_lateral_ventricle,wm_hypointensities,HippocampusOccupancyScore,HippocampusAsymmetryIndex,subject,sort_key
1,1.783869e+06,1.503270e+07,252787.557085,15076.564230,1219.707938,12931.785339,55145.136183,7858.472599,4185.628786,5293.175730,...,25875.232678,112256.005265,31510.722390,2368.757879,33879.480269,3632.712814,0.779730,0.005155,OAS2_0171_MR1_mpr-1,171
3,1.620240e+06,1.519633e+07,202643.703647,30157.930459,1520.861922,9846.157497,45718.810664,6624.015662,3024.230845,4675.089761,...,19451.186006,92990.039249,63302.018766,3342.534829,66644.553595,9640.357507,0.666633,0.026868,OAS2_0185_MR1_mpr-1,185
5,1.440649e+06,1.537592e+07,161264.528761,19763.658990,1865.233905,10172.693480,45737.332663,6079.331689,3181.324837,4389.713776,...,19685.454994,90936.155354,33421.918292,3389.868827,36811.787119,2378.704878,0.642860,0.033286,OAS2_0165_MR1_mpr-1,165
19,1.608101e+06,1.520847e+07,203166.778620,36894.450115,1985.969899,12120.933381,43074.280799,6884.009648,3661.524813,3846.058804,...,24280.625760,87000.230555,64950.133682,3683.819812,68633.953494,44501.503726,0.641812,0.032503,OAS2_0164_MR1_mpr-1,164
21,1.322464e+06,1.549410e+07,185442.597526,9640.700507,397.536980,13101.913331,57709.747052,6395.234673,3492.425822,3711.259810,...,26256.648659,116668.014039,17920.377084,843.093957,18763.471041,1097.599944,0.886913,0.011979,OAS2_0186_MR1_mpr-1,186
33,1.534214e+06,1.528235e+07,210800.243230,31177.326407,1450.203926,10991.777438,47181.705590,6747.495655,3268.446833,4255.943783,...,21608.998896,96056.802092,77831.841024,3376.148828,81207.989851,2674.027863,0.616131,0.008991,OAS2_0182_MR1_mpr-1,182
47,1.385521e+06,1.543105e+07,196618.222955,20922.312931,775.865960,11935.713390,49614.604465,6318.059677,3262.958833,4306.021780,...,23918.760778,98781.936953,42469.228830,1709.511913,44178.740743,3916.716800,0.785841,0.015159,OAS2_0183_MR1_mpr-1,183
55,1.511523e+06,1.530504e+07,202602.200649,20756.300940,1112.691943,12812.421345,53764.904253,7032.528641,3387.467827,4933.368748,...,24753.279735,106260.365571,44720.337715,2179.078889,46899.416604,1981.853899,0.765861,0.020115,OAS2_0061_MR1_mpr-1,61
59,1.306255e+06,1.551031e+07,188102.219390,13050.463333,671.593966,13358.477318,46369.481631,6754.012655,3360.027828,4556.411767,...,26014.147671,93869.148204,29771.369479,1312.317933,31083.687412,3055.100844,0.838066,0.031776,OAS2_0029_MR1_mpr-1,29
72,1.556886e+06,1.525968e+07,223946.746559,16883.831137,1236.171937,12453.986364,48029.258546,6397.978673,2881.542853,4294.702781,...,24723.781737,97637.003012,36628.625129,2554.320870,39182.945998,2373.559879,0.706285,0.037649,OAS2_0108_MR1_mpr-1,108
